In [3]:
import numpy as np
# import data as data
from gensim.models import Word2Vec
import pandas as pd

import warnings

df = pd.read_csv("autotagger.tsv", sep='\t')
t = df[df['description'].isna() == False]
t.isna().sum()
t = t[t['kw_name'] != 'ToBeRemoved']
data = t.copy()
data.shape
data.reset_index(drop=True, inplace=True)
data['product_name'] = data['product_name'].apply(lambda x: x.strip())

data['desc_and_productname'] = data['product_name'] + ' ' + data['description']

from gensim.parsing.preprocessing import preprocess_string

preprocess_string('Dispose 111/. / of unwanted played rubbish without having')

# https://radimrehurek.com/gensim/parsing/preprocessing.html

all_description = []
for description in data['desc_and_productname'].values:
    all_description.append(preprocess_string(description))

data['desc_processed'] = all_description

data = data[data['desc_processed'].apply(lambda x: len(x) >= 1)]
data['kw_name_bert'] = data['kw_name']

all_description = data['desc_processed']

model = Word2Vec(all_description,
                 min_count=1,  # word frequency
                 size=10,  # dimention of word embeddings # vector_size
                 workers=8,  # Number of processors
                 sg=0,  # cbow
                 window=5,  # Context window
                 )

model.wv.save_word2vec_format('w2vec_10d.txt')

# How to load:
from gensim.models import KeyedVectors

w2v = KeyedVectors.load_word2vec_format('w2vec_10d.txt')
# w2v.wv.vocab

vector = []
for line in all_description:
    try:
        x = np.zeros(10)
        for word in line:
            x = x + w2v[word]
        vector.append(x / len(line))
    except:
        print(line)

data['vector'] = vector
data.shape

df_train = data[data['kw_name'].isna() == False]
df_test = data[data['kw_name'].isna() == True]
c = df_train[['kw_name', 'vector']]
c = c[~c['kw_name'].apply(tuple).duplicated()]
ProductName_vector_dict = dict(data[['product_name', 'vector']].values)

import numpy as np
from numpy.linalg import norm


# define the function
def c_sim(ser1, ser2):
    """
    Function to compute the cosine similarity
    """
    # compute cosine similarity
    cos = np.dot(ser1, ser2) / (norm(ser1) * norm(ser2))

    # return cosine similarity accurate to 3 decimal places
    return round(cos, 3)


# ProductName_vector_dict.get('hahahaha', np.random.random(10))


def suggest_tags(ProductName):
    name = preprocess_string(ProductName)
    
    vector = []
    
    x = np.zeros(10)
    for word in name:
        x = x + (w2v[word] if word in w2v.vocab else np.zeros(10))
    vectorX = x / len(name)
    
    cs_store = []
    for _, line in c.iterrows():
        vectorY = line['vector']
        cs_store.append((c_sim(vectorX, vectorY), line['kw_name']))
    return [tag for _, tag in sorted(cs_store)[:3]]

In [4]:
df

,product_id,product_name,kw_name,description
0,126879,Addis Flip Top 46L Black Bin,NaN,Dispose of unwanted rubbish without having to ...
1,127376,Arnott's Tim Tam Chewy Caramel 200g,NaN,NaN
2,128411,Beeno Flatties Chicken 120g,NaN,NaN
3,138386,Beyers Amarula Chocolate 110g,NaN,NaN
4,138490,Bob Martin 2 In 1 Shampoo And Conditioner 400ml,NaN,NaN
...,...,...,...,...
13630,93196,Zonnebloem Pinotage 750ml,Red Wine,NaN
13631,93196,Zonnebloem Pinotage 750ml,Wine,NaN
13632,93196,Zonnebloem Pinotage 750ml,wines,NaN
13633,115640,Zooty Groovy 60ml,NaN,NaN


In [5]:
suggest_tags('Flip foo foo')

['pumo', 'bottled water', 'sparkling']